In [9]:
import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import subprocess

# quadra(しぶんぎ), perseids(ペルセウス),orionid(オリオン),draconids(りゅう座),leonid(しし),geminid(ふたご)
#perseids_ζ(ペルセ_昼間)，Aquariids_δ(みずがめ_δ)，Aquariids_η(みずがめ_η)，monoceros(12月いっかくじゅう)
meteorstream = np.array(["quadra", "perseids", "orionid", "draconids", "leonid", "geminid",\
                         "perseids_zeta","aquariids_delta","aquariids_ita","monoceros","Lyrid"])
a_1 = np.array([3.14, 71.4, 9.68, 3.02, 10.1, 1.372, 1.492, 1.971, 11.6, 50.7,46.05]) #Semi-major axis[au]
q_1 = np.array([0.979, 0.953, 0.571, 0.996, 0.9853, 0.141, 0.365, 0.067, 0.545, 0.193, 0.921]) #Perihelion distance[au]
i_1 = np.array([72.0, 113.22, 164.2, 31.4, 162.36, 24.02, 6.5, 30.8, 165.1, 35.2, 79,7]) #Inclination[deg]
omega_1 = np.array([172.0, 151.3, 82.8, 172.9, 173.5, 324.42, 60.5, 154.5, 91.9, 128.1, 213.8]) #Argument of perihelion[deg]
Omega_1 = np.array([283.3, 140.19, 208.6, 196.4, 236.15, 261.49, 81.5, 307.2, 45.6, 80.2, 32.5]) #Longitude of ascending node[deg]
e_1 = 1 - q_1/a_1 #Eccentricity[-]
#Dsh  添え字1がカタログ軌道、添え字2がmuレーダーの軌道

#df = pd.read_csv("../iccd_2018/MU_data_2009-2015.csv") #どちらを使うか
df = pd.read_csv("../Tomo-e_pipeline/MU_orbit_data_20181023.csv")
columns = df.columns
array_df = df.values
a_2 = df["Semimajor_axis_(AU)"].values
q_2 = df["Perihelion_dist_(AU)"].values
i_2 = df["Inclination_(deg)"].values
omega_2 = df["Argument_of_periapsis_(deg)"].values
omega_2 = np.deg2rad(omega_2)
Omega_2_deg = df["Lon_of_asc_node_(deg)"].values
Omega_2 = np.deg2rad(Omega_2_deg)
e_2 = df["Eccentricity"].values
index = []
meteor_stream = 10 #流星群選ぶ
dsh_threshold = 0.2 # Dshの値
filename = str(meteorstream[meteor_stream]) + "(" + str(dsh_threshold) + ").csv"

for j in range(len(a_2)):
    iota_21 = np.arccos(np.cos(np.deg2rad(i_1[meteor_stream]))*np.cos(np.deg2rad(i_2[j])) +\
                        np.sin(np.deg2rad(i_1[meteor_stream]))*np.sin(np.deg2rad(i_2[j]))*\
                        np.cos(Omega_2[j] - np.deg2rad(Omega_1[meteor_stream])))
    if abs(Omega_2[j] - Omega_1[meteor_stream]) > 180:
        gamma = -1
    elif abs(Omega_2[j] - Omega_1[meteor_stream]) <= 180:
        gamma = 1
    pi_21 = omega_2[j] - np.deg2rad(omega_1[meteor_stream]) +\
                      2*gamma*np.arcsin(np.cos((np.deg2rad(i_2[j]) + np.deg2rad(i_1[meteor_stream]))/2)*\
                      np.sin((Omega_2[j] - np.deg2rad(Omega_1[meteor_stream]))/2)/np.cos(iota_21/2))
    Dsh2 = (np.deg2rad(e_2[j]) - np.deg2rad(e_1[meteor_stream]))**2 + (q_2[j] - q_1[meteor_stream])**2 +\
                     (2*np.sin(iota_21/2))**2 + (((e_1[meteor_stream] + e_2[j])/2)**2)*\
                     ((2*np.sin(pi_21/2))**2)
    Dsh = np.sqrt(Dsh2)
    if Dsh < dsh_threshold :
        index = np.hstack((index, j))
index = index.astype(np.int64)
array_df = array_df[index,:]

for i in  range(len(columns)):  # IDの前にある不要カラムを削除
    if columns[i] == "ID":
        delete_column = i
array_df = np.delete(array_df,np.arange(delete_column),1)
columns = np.delete(columns,np.arange(delete_column))

df_output = pd.DataFrame(data=array_df, columns=columns)
df_output.to_csv(filename)
print "Complete!"

Complete!


In [8]:
import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import subprocess

df = pd.read_csv("Lyrid(0.2).csv")
a_2 = df["Semimajor_axis_(AU)"].values
q_2 = df["Perihelion_dist_(AU)"].values
i_2 = df["Inclination_(deg)"].values
i_2 = np.deg2rad(i_2)
omega_2 = df["Argument_of_periapsis_(deg)"].values
omega_2 = np.deg2rad(omega_2)
Omega_2_deg = df["Lon_of_asc_node_(deg)"].values
Omega_2 = np.deg2rad(Omega_2_deg)
e_2 = df["Eccentricity"].values
f_angle = np.linspace(0, 6.3, num = 2000, endpoint = True)

for index in range(len(a_2)):
    r = []
    x = []
    y = []
    file_name = "meteor_orbit_" + str(index) + ".txt"
    for i in f_angle:
        r_2d = a_2[index]*(1. - e_2[index]**2.)/(1. + e_2[index]*np.cos(i))
        x_2d = r_2d*np.cos(i)
        y_2d = r_2d*np.sin(i)
        r = np.hstack((r,r_2d))
        x = np.hstack((x,x_2d))
        y = np.hstack((y,y_2d))
    X = x*(np.cos(Omega_2[index])*np.cos(omega_2[index]) - np.sin(Omega_2[index])*np.cos(i_2[index])*\
           np.sin(omega_2[index])) -y*(np.cos(Omega_2[index])*np.sin(omega_2[index]) +\
           np.sin(Omega_2[index])*np.cos(i_2[index])*np.cos(omega_2[index]))
    
    Y = x*(np.sin(Omega_2[index])*np.cos(omega_2[index]) + np.cos(Omega_2[index])*np.cos(i_2[index])*\
           np.sin(omega_2[index])) -y*(np.sin(Omega_2[index])*np.sin(omega_2[index]) - \
           np.cos(Omega_2[index])*np.cos(i_2[index])*np.cos(omega_2[index]))
    
    Z = x*(np.sin(i_2[index])*np.sin(omega_2[index])) + y*(np.sin(i_2[index])*np.cos(omega_2[index]))
    position = np.vstack((X,Y,Z))
    position = position.transpose()
    with open(file_name, mode='w') as f:
        for j in range(len(X)):
            a = str(position[j,0]) + " " + str(position[j,1]) + " " + str(position[j,2]) +"\n"
            f.write(a)
subprocess.call(["mkdir d_txt"], shell = True)
subprocess.call(["mv meteor_orbit_{0..1500}.txt d_txt/"], shell = True) #数が多いなら要調整
print "Complete!"

Complete!


In [1]:
import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import subprocess

planet = np.array(["mercury", "venus", "earth", "mars", "jupyter", "saturn"])
a_1 = np.array([0.38710, 0.72333, 1.0, 1.52368, 5.20260, 9.55491]) #Semi-major axis[au]
e_1 = np.array([0.20563, 0.00677, 0.01671, 0.09340, 0.04850, 0.05555]) #Eccentricity[-]
i_1 = np.array([7.0050, 3.3947, 0., 1.8497, 1.3033, 2.4889]) #Inclination[deg]
i_1 = np.deg2rad(i_1)
omega_1 = np.array([77.4561, 131.5637, 102.9373, 6.0602, 14.3312, 93.0572]) #Argument of perihelion[deg]
omega_1 = np.deg2rad(omega_1) #336.0602 
Omega_1 = np.array([48.3309, 76.6799, 0., 4.5581, 100.4644, 113.6655]) #Longitude of ascending node[deg]
Omega_1 = np.deg2rad(Omega_1) #49.5581
q_1 = a_1*(1. - e_1) #Perihelion distance[au]
f_angle = np.linspace(0, 6.3, num = 2000, endpoint = True)

for idx in range(len(a_1)):
    r = []
    x = []
    y = []
    file_name =  str(planet[idx]) + ".txt"
    for i in f_angle:
        r_2d = a_1[idx]*(1. - e_1[idx]**2.)/(1. + e_1[idx]*np.cos(i))
        x_2d = r_2d*np.cos(i)
        y_2d = r_2d*np.sin(i)
        r = np.hstack((r,r_2d))
        x = np.hstack((x,x_2d))
        y = np.hstack((y,y_2d))
    X = x*(np.cos(Omega_1[idx])*np.cos(omega_1[idx]) - np.sin(Omega_1[idx])*np.cos(i_1[idx])*\
           np.sin(omega_1[idx])) -y*(np.cos(Omega_1[idx])*np.sin(omega_1[idx]) +\
           np.sin(Omega_1[idx])*np.cos(i_1[idx])*np.cos(omega_1[idx]))
    
    Y = x*(np.sin(Omega_1[idx])*np.cos(omega_1[idx]) + np.cos(Omega_1[idx])*np.cos(i_1[idx])*\
           np.sin(omega_1[idx])) -y*(np.sin(Omega_1[idx])*np.sin(omega_1[idx]) - \
           np.cos(Omega_1[idx])*np.cos(i_1[idx])*np.cos(omega_1[idx]))
    
    Z = x*(np.sin(i_1[idx])*np.sin(omega_1[idx])) + y*(np.sin(i_1[idx])*np.cos(omega_1[idx]))
    position = np.vstack((X,Y,Z))
    position = position.transpose()
    with open(file_name, mode='w') as f:
        for j in range(len(X)):
            a = str(position[j,0]) + " " + str(position[j,1]) + " " + str(position[j,2]) +"\n"
            f.write(a)
subprocess.call(["mkdir d_planet"], shell = True)
subprocess.call(["mv mercury.txt d_planet/"], shell = True)
subprocess.call(["mv venus.txt d_planet/"], shell = True)
subprocess.call(["mv mars.txt d_planet/"], shell = True)
subprocess.call(["mv earth.txt d_planet/"], shell = True)
subprocess.call(["mv jupyter.txt d_planet/"], shell = True)
subprocess.call(["mv saturn.txt d_planet/"], shell = True)
print "Complete!"

Complete!
